In [ ]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt2
import seaborn as sns
import matplotlib as mpl
# import warnings
# warnings.filterwarnings("ignore")
import os
import skimage
from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial
import squidpy as sq
from PIL import Image
from PIL.TiffTags import TAGS
from skimage import io
import tifffile as tf
import scipy
print(skimage.__version__) # make sure version is 0.19.2
# from cellpose import plot
# from cellpose import models # my modfied code 
# from cellpose import io
#import cv2
import matplotlib.pyplot as plt

## 1. Load relevant ISS samples with deconvoluted cell types

In [ ]:
CM008 = sc.read('/nfs/team292/vl6/FetalReproductiveTract/ISS/CM008_annotated.h5ad')
CM008

In [ ]:
CM008.obs.head()

In [ ]:
CM015 = sc.read('/nfs/team292/vl6/FetalReproductiveTract/ISS/CM015_annotated.h5ad')
CM015

In [ ]:
CM015.obs.head()

In [ ]:
CM016 = sc.read('/nfs/team292/vl6/FetalReproductiveTract/ISS/CM016_annotated.h5ad')
CM016

In [ ]:
CM016.obs.head()

In [ ]:
CM013 = sc.read('/nfs/team292/vl6/FetalReproductiveTract/ISS/CM013_annotated.h5ad')
CM013

In [ ]:
CM013.obs.head()

In [ ]:
## Import annotations 

annots = pd.read_csv('/nfs/team292/vl6/FetalReproductiveTract/ISS/female_post10pcw_ISS_deconvolution.csv', index_col = 0)
annots.head()

In [ ]:
def add_celltype(adata_iss, rename, sample_name, annots):
    if rename:
        adata_iss.obs_names = [sample_name + '_' + i for i in adata_iss.obs_names.to_list()]
    adata_iss.obs['celltype'] = adata_iss.obs_names.map(annots['celltype'].to_dict())
    adata_iss.obs['celltype_fraction'] = adata_iss.obs_names.map(annots['celltype_fraction'].to_dict())
    return adata_iss

In [ ]:
CM008 = add_celltype(CM008, rename = True, sample_name = 'CM008', annots = annots)

In [ ]:
CM008.obs.head()

In [ ]:
sq.pl.spatial_scatter(
    CM008,
    spatial_key="spatial",
    color=['celltype'],
    shape=None,
    use_raw = False,
    size=20,
    cmap = 'jet',
    figsize=(10,10),
)

In [ ]:
sq.pl.spatial_scatter(
    CM008,
    spatial_key="spatial",
    color=['celltype_fraction'],
    shape=None,
    use_raw = False,
    size=20,
    cmap = 'jet',
    figsize=(10,10),
)

In [ ]:
CM015 = add_celltype(CM015, rename = True, sample_name = 'CM015', annots = annots)

In [ ]:
sq.pl.spatial_scatter(
    CM015,
    spatial_key="spatial",
    color=['celltype_fraction'],
    shape=None,
    use_raw = False,
    size=20,
    cmap = 'jet',
    figsize=(10,10),
)

In [ ]:
CM016 = add_celltype(CM016, rename = True, sample_name = 'CM016', annots = annots)

In [ ]:
sq.pl.spatial_scatter(
    CM016,
    spatial_key="spatial",
    color=['celltype_fraction'],
    shape=None,
    use_raw = False,
    size=20,
    cmap = 'jet',
    figsize=(10,10),
)

In [ ]:
CM013 = add_celltype(CM013, rename = True, sample_name = 'CM013', annots = annots)

In [ ]:
sq.pl.spatial_scatter(
    CM013,
    spatial_key="spatial",
    color=['celltype_fraction'],
    shape=None,
    use_raw = False,
    size=20,
    cmap = 'jet',
    figsize=(10,10),
)

In [ ]:
def transpose_axis(orig_axis, new_start, new_end):

    # Apply linear transformation
    dest_axis = ((orig_axis - (-1)) / (1 - (-1))) * (new_end - new_start) + new_start
    
    return dest_axis


## 2. Compute uterovaginal axis

In [ ]:
def build_uterovaginal_axis(adata, sample, min_fract, which_axis,
                            start_feature_segment1, end_feature_segment1, 
                            start_feature_segment2 = None, end_feature_segment2 = None, 
                            start_weight_segment1 = 1, end_weight_segment1 = 1,
                            start_weight_segment2 = 1, end_weight_segment2 = 1,
                            excluded_regions = ['UterusTransverse', 'Ovary', 'Epoophron', 'VaginalPlate',
                                                'Unassigned', 'FallopianTube', 'LowerTract', 
                                               'Unassigned', 'Background'], is_stitched = False, sample1 = None, sample2 = None):
    
    print('Is the sample stitched? {}'.format(is_stitched))
    
    if not is_stitched:
        
        # Select slide from spatial anndata object
        #adata_sample = select_slide(adata, sample, batch_key = 'sample')
        adata_sample = adata.copy()
        
        # Remove excluded regions 
        adata_sample = adata_sample[[i not in excluded_regions for i in adata_sample.obs['broad_anatomy']]]
        print(adata_sample.shape)
        adata_sample = adata_sample[[i > min_fract for i in adata_sample.obs['celltype_fraction']]]
        print(adata_sample.shape)
        
        if which_axis == 'uterocervical':
            print('Building uterocervical axis for sample {} from {} to {}'.format(sample, start_feature_segment1, 
                             end_feature_segment1))
        
            # Build uterocervical axis (first segment from uterine fundus to uterine bottom)
            df1 = adata_sample.obs[['L2_dist_longitudinal_anatomy_annots_' + start_feature_segment1, 'L2_dist_longitudinal_anatomy_annots_' + end_feature_segment1, 
               'broad_anatomy_annots', 'histology_annots']]
            print(df1.head())
            df1.dropna(inplace=True)
            uterovaginal_axis_segment1 = [( start_weight_segment1*int(s) - end_weight_segment1*int(e) ) / ( start_weight_segment1*int(s) + end_weight_segment1*int(e) ) for s,e in zip(df1['L2_dist_longitudinal_anatomy_annots_' + start_feature_segment1], df1['L2_dist_longitudinal_anatomy_annots_' + end_feature_segment1])]
        
            df1['UteroCervical_Axis'] = uterovaginal_axis_segment1
            print(df1['UteroCervical_Axis'].hist(bins=20))
        
            # Add uterocervical axis to slide 
            adata_sample.obs['UteroCervical_Axis'] = adata_sample.obs_names.map(df1['UteroCervical_Axis'].to_dict())
            sc.pl.spatial(adata_sample[[i in ['Epithelium', 'Mesenchyme', 
                                          'SmoothMuscle', 'Ligament'] for i in adata_sample.obs['histology_annots']]],
                                      color="UteroCervical_Axis", title = 'UteroCervical Axis', cmap = 'jet')
            
        elif which_axis == 'uterovaginal':
            print('Building uterovaginal axis for sample {} from {} to {}'.format(sample, start_feature_segment1, 
                            end_feature_segment1))
            
            # Build uterocervical axis (first segment from uterine fundus to uterine bottom)
            df1 = adata_sample.obs[['L2_dist_longitudinal_anatomy_' + start_feature_segment1, 'L2_dist_longitudinal_anatomy_' + end_feature_segment1, 
               'broad_anatomy']]
            df1.dropna(inplace=True)
            uterovaginal_axis_segment1 = [( start_weight_segment1*int(s) - end_weight_segment1*int(e) ) / ( start_weight_segment1*int(s) + end_weight_segment1*int(e) ) for s,e in zip(df1['L2_dist_longitudinal_anatomy_' + start_feature_segment1], df1['L2_dist_longitudinal_anatomy_' + end_feature_segment1])]
        
            df1['UteroVaginal_Axis'] = uterovaginal_axis_segment1
            print(df1['UteroVaginal_Axis'].hist(bins=20))
            df1['UteroVaginal_Axis_forPlotting'] = transpose_axis(df1['UteroVaginal_Axis'], 0, 4)
        
            # Add uterocervical axis to slide 
            adata_sample.obs['UteroVaginal_Axis'] = adata_sample.obs_names.map(df1['UteroVaginal_Axis'].to_dict())
            adata_sample.obs['UteroVaginal_Axis_forPlotting'] = adata_sample.obs_names.map(df1['UteroVaginal_Axis_forPlotting'].to_dict())
            sq.pl.spatial_scatter(adata_sample, 
                          spatial_key="spatial",
                    color=['UteroVaginal_Axis_forPlotting'],
                    shape=None,
                    use_raw = False,
                    size=20,
                        cmap = 'jet',
                    figsize=(10,10),)

        
        
        return adata_sample 
    
    else: 
        
        
        # Read stitched sample directly 
        adata_stitched = sc.read(path_to_reptract + sample1 + '_' + sample2 + '/' + sample1 + '_' + sample2 + '_annotated.h5ad')
        
        # Add a prefix to the distances computed on the joint object 
        for col in adata_stitched.obs.columns:
            if start_feature_segment2 != None:
                if col in [ 'L2_dist_longitudinal_anatomy_' + start_feature_segment1, 
                       'L2_dist_longitudinal_anatomy_' + end_feature_segment1, 
                       'L2_dist_longitudinal_anatomy_' + start_feature_segment2,
                       'L2_dist_longitudinal_anatomy_' + end_feature_segment2,
                       'broad_anatomy_annots', 'longitudinal_anatomy']:
                    adata_stitched.obs.rename(columns={col : 'joint_' + col}, inplace = True)
            else:
                if col in [ 'L2_dist_longitudinal_anatomy_' + start_feature_segment1, 
                       'L2_dist_longitudinal_anatomy_' + end_feature_segment1, 
                       'broad_anatomy_annots', 'longitudinal_anatomy']:
                    adata_stitched.obs.rename(columns={col : 'joint_' + col}, inplace = True)
                
        # Concatenate 
        adata_separate = select_slide(adata, sample1, batch_key = 'sample').concatenate(select_slide(adata, sample2, batch_key = 'sample'), index_unique = None)
        adata_separate = adata_separate[[i > min_cells for i in adata_separate.obs['tot_cell_abundance']]]
        
        # Import distances 
        for col in adata_stitched.obs.columns:
            if col.startswith('joint') or col == 'overlaps_with':
                adata_separate.obs[col] = adata_separate.obs_names.map(adata_stitched.obs[col].to_dict())
            
        # Create a dictionary mapping the index of the missing value to the index of the replacement value
        replacement_dict = adata_separate.obs['overlaps_with'].dropna().to_dict()
        replacement_dict = {v:k for k,v in replacement_dict.items()}
        
        to_remove = []
        for col in adata_separate.obs.columns:
            if col.startswith('joint'):
                for spot_id in adata_separate.obs_names:
                    value = adata_separate.obs.at[spot_id, col]
                    if isinstance(value, (float, np.float64)) and np.isnan(value):
                        if spot_id in list(replacement_dict.keys()):
                            overlapping_spot = replacement_dict[spot_id]
                            if overlapping_spot in adata_separate.obs_names:
                                adata_separate.obs.at[spot_id, col] = adata_separate.obs.at[overlapping_spot, col]
                        elif spot_id not in list(replacement_dict.keys()):
                            to_remove.append(spot_id)
        remove = list(np.unique(to_remove))
        
        # create a boolean mask of the observations to keep
        mask = ~adata_separate.obs_names.isin(remove)

        # select only the rows that match the mask
        adata_separate = adata_separate[mask] 
        
        adata_separate = adata_separate[[i not in excluded_regions for i in adata_separate.obs['broad_anatomy_annots']]]

        
        if which_axis == 'uterocervical':
            print('Building uterocervical axis for samples {} and {} from {} to {}'.format(sample1, sample2, start_feature_segment1, 
                             end_feature_segment1))
            

        
            # Build uterocervical axis (first segment from uterine fundus to uterine bottom)
            df1 = adata_separate.obs[['joint_L2_dist_longitudinal_anatomy_' + start_feature_segment1, 'joint_L2_dist_longitudinal_anatomy_' + end_feature_segment1, 
               'broad_anatomy_annots', 'histology_annots', 'joint_longitudinal_anatomy']]
            df1.dropna(inplace=True)
            uterovaginal_axis_segment1 = [( start_weight_segment1*int(s) - end_weight_segment1*int(e) ) / ( start_weight_segment1*int(s) + end_weight_segment1*int(e) ) for s,e in zip(df1['joint_L2_dist_longitudinal_anatomy_' + start_feature_segment1], df1['joint_L2_dist_longitudinal_anatomy_' + end_feature_segment1])]
        
            df1['UteroCervical_Axis'] = uterovaginal_axis_segment1
            print(df1['UteroCervical_Axis'].hist(bins=20, color = 'navy'))
        
            # Add uterocervical axis to slide 
            adata_separate.obs['UteroCervical_Axis'] = adata_separate.obs_names.map(df1['UteroCervical_Axis'].to_dict())
            
        elif which_axis == 'uterovaginal':
            print('Building uterovaginal axis for samples {} and {} from {} to {}'.format(sample1, sample2, start_feature_segment1, 
                            end_feature_segment2))
        
            # Build cervicovaginal axis (second segment from uterine fundus to lower tract)
            df2 = adata_separate.obs[['joint_L2_dist_longitudinal_anatomy_' + start_feature_segment1, 'joint_L2_dist_longitudinal_anatomy_' + end_feature_segment2, 
                   'broad_anatomy_annots', 'histology_annots', 'joint_longitudinal_anatomy']]
            print(df2.head())
            df2.dropna(inplace=True)
            uterovaginal_axis = [( start_weight_segment1*int(s) - end_weight_segment2*int(e) ) / ( start_weight_segment1*int(s) + end_weight_segment2*int(e) ) for s,e in zip(df2['joint_L2_dist_longitudinal_anatomy_' + start_feature_segment1], df2['joint_L2_dist_longitudinal_anatomy_' + end_feature_segment2])]

            df2['UteroVaginal_Axis'] = uterovaginal_axis
            plt.figure(figsize=(10, 5))  # Set the figure size
            plt.subplot(1, 2, 1)
            plt.hist(df2['UteroVaginal_Axis'], bins=20, alpha=0.7, color='blue', label='UteroVaginal_Axis')
            adata_separate.obs['UteroVaginal_Axis'] = adata_separate.obs_names.map(df2['UteroVaginal_Axis'].to_dict())

            # Recode uterovaginal axis so that 'UterineBottom' is found at position +1
            # Find the mean coordinate along the uterovaginal axis where the longitudinal anatomical annotation is uterine bottom 
            mean_coordinate_uterinebottom = df2.loc[df2['joint_longitudinal_anatomy'] == 'UterineBottom', 'UteroVaginal_Axis'].mean()
            print(mean_coordinate_uterinebottom)
            
            b = (1-mean_coordinate_uterinebottom)/(1+mean_coordinate_uterinebottom)
            a = b+1
            df2['UteroVaginal_Axis_Recoded'] = a * df2['UteroVaginal_Axis'] + b
            plt.subplot(1, 2, 2)
            plt.hist(df2['UteroVaginal_Axis_Recoded'], bins=20, alpha=0.7, color='blue', label='UteroVaginal_Axis_Recoded')
            plt.tight_layout()  # Adjust layout to prevent overlapping
            plt.show()
            # Add cervicovaginal axis to slide 
            adata_separate.obs['UteroVaginal_Axis_Recoded'] = adata_separate.obs_names.map(df2['UteroVaginal_Axis_Recoded'].to_dict())
            
#             df2['UteroVaginal_Axis_Recoded'] = transpose_axis(df2['UteroVaginal_Axis'], -1, 1)
#             adata_separate.obs['UteroVaginal_Axis_Recoded'] = adata_separate.obs_names.map(df2['UteroVaginal_Axis_Recoded'].to_dict())
            
        
#         df = adata_separate.obs[['joint_L2_dist_longitudinal_anatomy_annots_' + start_feature, 'joint_L2_dist_longitudinal_anatomy_annots_' + end_feature, 
#                'joint_broad_anatomy_annots']]
        #print(df.head())
#         uterovaginal_axis = [( start_weight*int(s) - end_weight*int(e) ) / ( start_weight*int(s) + end_weight*int(e) ) for s,e in zip(df['joint_L2_dist_longitudinal_anatomy_annots_' + start_feature], df['joint_L2_dist_longitudinal_anatomy_annots_' + end_feature])]
#         df['UteroVaginal_Axis'] = uterovaginal_axis
        
#         print(df['UteroVaginal_Axis'].hist(bins=20))
        
        # Add axis to slide 
#         adata_separate.obs['UteroVaginal_Axis'] = adata_separate.obs_names.map(df['UteroVaginal_Axis'].to_dict())
        return adata_separate.obs
        


In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, 
                         fontsize=14, figsize=(10, 10), color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
CM008_axis = build_uterovaginal_axis(CM008, sample = 'CM008', min_fract = 0.4, which_axis = 'uterovaginal',
                                                 start_feature_segment1 = 'UterineFundus', end_feature_segment1 = 'EndodermalVaginalStart', 
                                                 )

In [ ]:
CM016_axis = build_uterovaginal_axis(CM016, sample = 'CM016', min_fract = 0.4, which_axis = 'uterovaginal',
                                                 start_feature_segment1 = 'UterineFundus', end_feature_segment1 = 'EndodermalVaginalStart', 
                                                 )

In [ ]:
CM013_axis_uv = build_uterovaginal_axis(CM013, sample = 'CM016', min_fract = 0.4, which_axis = 'uterovaginal',
                                                 start_feature_segment1 = 'UterineFundus', end_feature_segment1 = 'EndodermalVaginalStart', 
                                                 )

In [ ]:
for sample in [CM016_axis, CM008_axis, CM013_axis_uv]:
    sample.obs['FemaleReproductiveAxis'] = sample.obs['UteroVaginal_Axis_forPlotting']

In [ ]:
#CM016_axis.obs_names = ['CM016_' + i for i in CM016_axis.obs_names.to_list()]
CM016_axis.obs.head()

In [ ]:
#CM008_axis.obs_names = ['CM008_' + i for i in CM008_axis.obs_names.to_list()]
CM008_axis.obs.head()

In [ ]:
CM013_axis_uv.obs.head()

In [ ]:
def combine_axis(samples, axis_name, return_adata = False):
    
    # Merge anndata objects
    adata_all = samples[0].concatenate(samples[1:], index_unique = None)
    
    # Remove spots that don't have a value for the axis 
    adata_all_obs = adata_all.obs
    adata_all_obs = adata_all_obs[adata_all_obs[axis_name].notna()]
    adata_all = adata_all[adata_all_obs.index.to_list(), :]
    
    if return_adata:
        return adata_all
    else:
        return adata_all_obs

In [ ]:
uterovaginal_axis = combine_axis(samples = [
                                           CM016_axis, CM008_axis, CM013_axis_uv
                                           ], 
                                axis_name = 'FemaleReproductiveAxis', return_adata = True)

In [ ]:
uterovaginal_axis.obs.head()

## 3. Compute fallopian axis

In [ ]:
def build_fallopian_axis(adata, sample, min_fract, start_feature, end_feature, start_weight = 1, end_weight = 1,
                         excluded_regions = ['TransverseUterus', 'VaginalPlate', 'Unassigned', 'Uterus', 'Ovary', 
                                            'Epoophron', 'Background', 
                                            'UpperVagina', 'Cervix', 'LowerTract'], is_stitched = False, sample1 = None, sample2 = None):
    
    print('Is the sample stitched? {}'.format(is_stitched))
    
    if not is_stitched:
        
        print('Building uterovaginal axis for sample {} from {} (weight: {}) to {} (weight: {})'.format(sample, start_feature, 
                            start_weight, end_feature, end_weight))
        
        # Select slide from spatial anndata object
        #adata_sample = select_slide(adata, sample, batch_key = 'sample')
        adata_sample = adata.copy()
        
        # Remove excluded regions 
        adata_sample = adata_sample[[i not in excluded_regions for i in adata_sample.obs['broad_anatomy']]]
        adata_sample = adata_sample[[i > min_fract for i in adata_sample.obs['celltype_fraction']]]
            
        # Build axis 
        df = adata_sample.obs[['L2_dist_longitudinal_anatomy_' + start_feature, 'L2_dist_longitudinal_anatomy_' + end_feature, 
               'broad_anatomy']]
        print(df.head())
        print(df['L2_dist_longitudinal_anatomy_' + start_feature].isna().value_counts())
        fallopian_axis = [( start_weight*int(s) - end_weight*int(e) ) / ( start_weight*int(s) + end_weight*int(e) ) for s,e in zip(df['L2_dist_longitudinal_anatomy_' + start_feature], df['L2_dist_longitudinal_anatomy_' + end_feature])]
        df['Fallopian_Axis'] = fallopian_axis
        print(df['Fallopian_Axis'].hist(bins=20))
        df['Fallopian_Axis_forPlotting'] = transpose_axis(df['Fallopian_Axis'], -2, 0)
        
        # Add fallopian axis to slide 
        adata_sample.obs['Fallopian_Axis'] = adata_sample.obs_names.map(df['Fallopian_Axis'].to_dict())
        adata_sample.obs['Fallopian_Axis_forPlotting'] = adata_sample.obs_names.map(df['Fallopian_Axis_forPlotting'].to_dict())
        
        sq.pl.spatial_scatter(adata_sample, 
                          spatial_key="spatial",
                    color=['Fallopian_Axis_forPlotting'],
                    shape=None,
                    use_raw = False,
                    size=20,
                        cmap = 'jet',
                    figsize=(10,10),)
        
        return adata_sample 
    
    else: 
        
        print('Building uterovaginal axis for samples {} and {} from {} (weight: {}) to {} (weight: {})'.format(sample1, sample2, start_feature, 
                            start_weight, end_feature, end_weight))
        
        # Read stitched sample directly 
        adata_stitched = sc.read(path_to_reptract + sample1 + '_' + sample2 + '/' + sample1 + '_' + sample2 + '_annotated.h5ad')
        
        # Add a prefix to the distances computed on the joint object 
        for col in adata_stitched.obs.columns:
            if col in [ 'L2_dist_longitudinal_anatomy_annots_' + start_feature, 'L2_dist_longitudinal_anatomy_annots_' + end_feature, 'broad_anatomy_annots']:
                adata_stitched.obs.rename(columns={col : 'joint_' + col}, inplace = True)
                
        # Concatenate 
        adata_separate = select_slide(adata, sample1, batch_key = 'sample').concatenate(select_slide(adata, sample2, batch_key = 'sample'), index_unique = None)
        adata_separate = adata_separate[[i > min_cells for i in adata_separate.obs['tot_cell_abundance']]]
        
        # Import distances 
        for col in adata_stitched.obs.columns:
            if col.startswith('joint') or col == 'overlaps_with':
                adata_separate.obs[col] = adata_separate.obs_names.map(adata_stitched.obs[col].to_dict())
            
        # Create a dictionary mapping the index of the missing value to the index of the replacement value
        replacement_dict = adata_separate.obs['overlaps_with'].dropna().to_dict()
        replacement_dict = {v:k for k,v in replacement_dict.items()}
        
        to_remove = []
        for col in adata_separate.obs.columns:
            if col.startswith('joint'):
                for spot_id in adata_separate.obs_names:
                    value = adata_separate.obs.at[spot_id, col]
                    if isinstance(value, (float, np.float64)) and np.isnan(value):
                        if spot_id in list(replacement_dict.keys()):
                            overlapping_spot = replacement_dict[spot_id]
                            if overlapping_spot in adata_separate.obs_names:
                                adata_separate.obs.at[spot_id, col] = adata_separate.obs.at[overlapping_spot, col]
                        elif spot_id not in list(replacement_dict.keys()):
                            to_remove.append(spot_id)
        remove = list(np.unique(to_remove))
        
        # create a boolean mask of the observations to keep
        mask = ~adata_separate.obs_names.isin(remove)

        # select only the rows that match the mask
        adata_separate = adata_separate[mask]
        
        adata_separate = adata_separate[[i not in excluded_regions for i in adata_separate.obs['broad_anatomy_annots']]]
        
        df = adata_separate.obs[['joint_L2_dist_longitudinal_anatomy_' + start_feature, 'joint_L2_dist_longitudinal_anatomy_' + end_feature, 
               'joint_broad_anatomy_annots']]
        fallopian_axis = [( start_weight*int(s) - end_weight*int(e) ) / ( start_weight*int(s) + end_weight*int(e) ) for s,e in zip(df['joint_L2_dist_longitudinal_anatomy_' + start_feature], df['joint_L2_dist_longitudinal_anatomy_' + end_feature])]
        df['Fallopian_Axis'] = fallopian_axis
        
        print(df['Fallopian_Axis'].hist(bins=20))
        
        # Add axis to slide 
        adata_separate.obs['Fallopian_Axis'] = adata_separate.obs_names.map(df['Fallopian_Axis'].to_dict())
        return adata_separate
        


In [ ]:
CM015_axis = build_fallopian_axis(CM015, sample = 'CM015', min_fract = 0.4, 
                                                 start_feature = 'FallopianFimbriae', end_feature = 'FallopianUterineJunction', 
                                                 )

In [ ]:
CM016_axis_f = build_fallopian_axis(CM016, sample = 'CM016', min_fract = 0.4, 
                                                 start_feature = 'FallopianFimbriae', end_feature = 'FallopianUterineJunction', 
                                                 )

In [ ]:
CM013_axis_f = build_fallopian_axis(CM013, sample = 'CM013', min_fract = 0.4, 
                                                 start_feature = 'FallopianFimbriae', end_feature = 'FallopianUterineJunction', 
                                                 )

In [ ]:
for sample in [CM015_axis, CM016_axis_f, CM013_axis_f]:
    sample.obs['FemaleReproductiveAxis'] = sample.obs['Fallopian_Axis_forPlotting']

In [ ]:
#CM015_axis.obs_names = ['CM015_' + i for i in CM015_axis.obs_names.to_list()]
CM015_axis.obs.head()

In [ ]:
#CM016_axis_f.obs_names = ['CM016_' + i for i in CM016_axis_f.obs_names.to_list()]
CM016_axis_f.obs.head()

In [ ]:
CM013_axis_f.obs.head()

In [ ]:
fallopian_axis = combine_axis(samples = [
                                           CM015_axis, CM016_axis_f, CM013_axis_f
                                           ], 
                                axis_name = 'FemaleReproductiveAxis', return_adata = True)

In [ ]:
fallopian_axis

## 4. Concatenate fallopian and uterovaginal axis

In [ ]:
fallopian_axis = fallopian_axis.obs.copy()

In [ ]:
fallopian_axis['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
uterovaginal_axis = uterovaginal_axis.obs.copy()

In [ ]:
uterovaginal_axis['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
axis = fallopian_axis.append(uterovaginal_axis)

In [ ]:
axis['FemaleReproductiveAxis'].hist(bins = 10)

## 5. Transfer axis values to anndata objects 

In [ ]:
axis

In [ ]:
CM008_axis.shape, CM008.shape 

In [ ]:
CM015_axis.shape, CM015.shape 

In [ ]:
CM016_axis.shape, CM016.shape 

In [ ]:
CM016_axis_f.shape, CM016.shape 

In [ ]:
for sample in [CM008_axis, CM015_axis, CM016_axis, CM016_axis_f, CM013_axis_uv, CM013_axis_f]:
    sample.obs['FemaleReproductiveAxis'] = sample.obs_names.map(axis['FemaleReproductiveAxis'].to_dict())

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
sq.pl.spatial_scatter(CM008_axis, color="FemaleReproductiveAxis", 
                spatial_key="spatial",
                shape=None,
                use_raw = False,
                size=0.6,
                figsize=(8,8), cmap = 'jet', 
                     save = 'CM008_femalereproductiveaxis.pdf')

In [ ]:
sq.pl.spatial_scatter(CM015_axis, color="FemaleReproductiveAxis", 
                spatial_key="spatial",
                shape=None,
                use_raw = False,
                size=0.6,
                figsize=(8,8), cmap = 'jet', 
                     save = 'CM015_femalereproductiveaxis.pdf')

In [ ]:
sq.pl.spatial_scatter(CM016_axis, color="FemaleReproductiveAxis", 
                spatial_key="spatial",
                shape=None,
                use_raw = False,
                size=0.2,
                figsize=(8,8), cmap = 'jet', 
                     save = 'CM016_uterovaginal.pdf')

In [ ]:
sq.pl.spatial_scatter(CM016_axis_f, color="FemaleReproductiveAxis", 
                spatial_key="spatial",
                shape=None,
                use_raw = False,
                size=0.2,
                figsize=(8,8), cmap = 'jet', 
                     save = 'CM016_fallopian.pdf')

In [ ]:
sq.pl.spatial_scatter(CM013_axis_f, color="FemaleReproductiveAxis", 
                spatial_key="spatial",
                shape=None,
                use_raw = False,
                size=0.2,
                figsize=(8,8), cmap = 'jet', 
                     save = 'CM013_fallopian.pdf')

In [ ]:
sq.pl.spatial_scatter(CM013_axis_uv, color="FemaleReproductiveAxis", 
                spatial_key="spatial",
                shape=None,
                use_raw = False,
                size=0.2,
                figsize=(8,8), cmap = 'jet', 
                     save = 'CM013_uterovaginal.pdf')

In [ ]:
total_axis_CM016 = CM016_axis.obs['FemaleReproductiveAxis'].append(CM016_axis_f.obs['FemaleReproductiveAxis'])

In [ ]:
CM016.obs['FemaleReproductiveAxis'] = CM016.obs_names.map(total_axis_CM016.to_dict())

In [ ]:
sq.pl.spatial_scatter(CM016, color="FemaleReproductiveAxis", 
                spatial_key="spatial",
                shape=None,
                use_raw = False,
                size=0.2,
                figsize=(8,8), cmap = 'jet', 
                     save = 'CM016_femalereproductiveaxis.pdf')

In [ ]:
total_axis_CM013 = CM013_axis_uv.obs['FemaleReproductiveAxis'].append(CM013_axis_f.obs['FemaleReproductiveAxis'])

In [ ]:
CM013.obs['FemaleReproductiveAxis'] = CM013.obs_names.map(total_axis_CM013.to_dict())

In [ ]:
sq.pl.spatial_scatter(CM013, color="FemaleReproductiveAxis", 
                spatial_key="spatial",
                shape=None,
                use_raw = False,
                size=0.2,
                figsize=(8,8), cmap = 'jet', 
                     save = 'CM013_femalereproductiveaxis.pdf')

## 6. Merge ISS samples together 

In [ ]:
CM016_cells = CM016_axis.obs_names.to_list()

In [ ]:
CM016_axis.shape

In [ ]:
CM016_axis_f.shape

In [ ]:
CM016_cells.extend(CM016_axis_f.obs_names.to_list())

In [ ]:
len(CM016_cells)

In [ ]:
CM016 = CM016[[i in CM016_cells for i in CM016.obs_names.to_list()]]

In [ ]:
CM016

In [ ]:
CM013_cells = CM013_axis_uv.obs_names.to_list()
print(CM013_axis_uv.shape)
CM013_cells.extend(CM013_axis_f.obs_names.to_list())
print(len(CM013_cells))

In [ ]:
CM013 = CM013[[i in CM013_cells for i in CM013.obs_names.to_list()]]

In [ ]:
CM013

In [ ]:
adata_iss = sc.concat(
    [CM008_axis, CM015_axis, CM016, CM013],
    join = "outer",
    uns_merge="unique",

    index_unique=None,
)
adata_iss.var_names_make_unique()

In [ ]:
adata_iss.X[40:50, 60:70]

In [ ]:
adata_iss.obs['FemaleReproductiveAxis'].hist()

In [ ]:
np.sum(np.isnan(adata_iss.obs['FemaleReproductiveAxis']))

In [ ]:
adata_iss.write('/nfs/team292/vl6/FetalReproductiveTract/ISS/CM008_CM015_CM016_CM013_with_axis.h5ad')

In [ ]:
adata_iss

### *Notebook for imputing Müllerian longitudinal axis scRNA-seq* 

## 7. Convert scRNA-seq with ISS axis to RDS

In [ ]:
mese_raw = sc.read('/nfs/team292/vl6/FetalReproductiveTract/mese_mullerian_withpseudospace.h5ad')

In [ ]:
mese_raw.var.head()

In [ ]:
mese_raw.obs['FemaleReproductiveAxis'].hist()

In [ ]:
mese_raw.shape

In [ ]:
def bin_axis(df, n_bins = 10, axis_name = 'FemaleReproductiveAxis'):
    max_val = np.nanquantile(df[axis_name],0.9999)
    min_val = np.nanquantile(df[axis_name],0.0001)

    incr_uva = (max_val-min_val)/n_bins
    uva_bins = []
    for i in range(0,n_bins+1):
        uva_bins.append(min_val+(i*incr_uva))

    df['binned_' + axis_name + '_' + str(n_bins) + '_bins'] = pd.cut(df[axis_name], bins = uva_bins, labels=uva_bins[:-1])
    return df

In [ ]:
axis = bin_axis(mese_raw.obs, n_bins = 10, axis_name = 'FemaleReproductiveAxis')

In [ ]:
axis['binned_FemaleReproductiveAxis_10_bins'] = axis['binned_FemaleReproductiveAxis_10_bins'].astype(float)

In [ ]:
axis['binned_FemaleReproductiveAxis_10_bins'] = np.round(axis['binned_FemaleReproductiveAxis_10_bins'], 2)

In [ ]:
axis['binned_FemaleReproductiveAxis_10_bins'].value_counts()

### Downsample per axis bin 

In [ ]:
mese_raw.obs['binned_FemaleReproductiveAxis_10_bins'] = mese_raw.obs_names.map(axis['binned_FemaleReproductiveAxis_10_bins'].to_dict())

In [ ]:
# Random downsampling per cell type 
import random
from itertools import chain
def downsample(adata, labels, n): 
    
    myindex = adata.obs[labels].value_counts().index 
    myvalues = adata.obs[labels].value_counts().values
    clusters = pd.Series(myvalues, index = myindex)
    
    # Find clusters with > n cells 
    cl2downsample = clusters.index[ clusters.values > n ]

    # save all barcode ids from small clusters
    holder = []
    holder.append( adata.obs_names[[ i not in cl2downsample for i in adata.obs[labels] ]] ) 

    # randomly sample n cells in the cl2downsample
    for cl in cl2downsample:
        print(cl)
        cl_sample = adata[[ i == cl for i in adata.obs[labels]]].obs_names
        
        cl_downsample = random.sample(set(cl_sample), n )
        holder.append(cl_downsample)
    
    # samples to include
    samples = list(chain(*holder))

    # Filter adata_count
    adata = adata[[ i in samples for i in adata.obs_names ]]
    return adata

In [ ]:
mese_raw = downsample(mese_raw, 'binned_FemaleReproductiveAxis_10_bins', 1742)

In [ ]:
mese_raw.shape

In [ ]:
mese_raw

In [ ]:
mese_raw.X.toarray()[30:40, 30:40]

In [ ]:
mese_raw_forR = mese_raw.copy()

In [ ]:
mito_genes = [name for name in mese_raw_forR.var_names if name.startswith('MT-')]


In [ ]:
# remove mito genes
non_mito_genes = [name for name in mese_raw_forR.var_names if not name.startswith('MT')]
mese_raw_forR = mese_raw_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in mese_raw_forR.var_names if not name.startswith('RP')]
mese_raw_forR = mese_raw_forR[:, non_ribo_genes]

# remove heat shock protein genes
non_hps_genes = [name for name in mese_raw_forR.var_names if not name.startswith('HSP')]
mese_raw_forR = mese_raw_forR[:, non_hps_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in mese_raw_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
mese_raw_forR = mese_raw_forR[:, non_haemo_genes]

sc.pp.filter_genes(mese_raw_forR, min_cells=10)
sc.pp.filter_genes(mese_raw_forR, min_counts=1)

In [ ]:
'HSP1' in mese_raw_forR.var_names.to_list()

In [ ]:
mese_raw_forR.shape

In [ ]:
for col in mese_raw_forR.obs.columns:
    if col not in [ 'FemaleReproductiveAxis', 'sample']:
        del mese_raw_forR.obs[col]

In [ ]:
del mese_raw_forR.obsm

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R -i mese_raw_forR
mese_raw_forR

In [ ]:
%%R 
saveRDS(mese_raw_forR, '/lustre/scratch126/cellgen/team292/vl6/mese_mullerian_withpseudospace.rds')

In [ ]:
mese_raw_forR.obs['FemaleReproductiveAxis'].hist()

In [ ]:
mese_raw.obs['binned_FemaleReproductiveAxis_10_bins'].value_counts()

In [ ]:
mese_raw.write('/nfs/team292/vl6/FetalReproductiveTract/mese_mullerian_withpseudospace_downsampled.h5ad')

In [ ]:
epi_raw = sc.read('/nfs/team292/vl6/FetalReproductiveTract/epi_mullerian_withpseudospace.h5ad')

In [ ]:
epi_raw.shape

In [ ]:
epi_raw.obs['FemaleReproductiveAxis'].hist()

In [ ]:
axis = bin_axis(epi_raw.obs, n_bins = 10, axis_name = 'FemaleReproductiveAxis')

In [ ]:
axis['binned_FemaleReproductiveAxis_10_bins'] = axis['binned_FemaleReproductiveAxis_10_bins'].astype(float)

In [ ]:
axis['binned_FemaleReproductiveAxis_10_bins'] = np.round(axis['binned_FemaleReproductiveAxis_10_bins'], 2)

In [ ]:
axis['binned_FemaleReproductiveAxis_10_bins'].value_counts()

In [ ]:
epi_raw.obs['binned_FemaleReproductiveAxis_10_bins'] = epi_raw.obs_names.map(axis['binned_FemaleReproductiveAxis_10_bins'].to_dict())


In [ ]:
epi_raw = downsample(epi_raw, 'binned_FemaleReproductiveAxis_10_bins', 500)

In [ ]:
epi_raw.shape

In [ ]:
epi_raw.obs['FemaleReproductiveAxis'].hist()

In [ ]:
epi_raw.X.toarray()[30:40, 30:40]

In [ ]:
epi_raw_forR = epi_raw.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in epi_raw_forR.var_names if not name.startswith('MT-')]
epi_raw_forR = epi_raw_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in epi_raw_forR.var_names if not name.startswith('RP')]
epi_raw_forR = epi_raw_forR[:, non_ribo_genes]

# remove heat shock protein genes
non_hps_genes = [name for name in epi_raw_forR.var_names if not name.startswith('HSP')]
epi_raw_forR = epi_raw_forR[:, non_hps_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in epi_raw_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
epi_raw_forR = epi_raw_forR[:, non_haemo_genes]

sc.pp.filter_genes(epi_raw_forR, min_cells=10)
sc.pp.filter_genes(epi_raw_forR, min_counts=1)

In [ ]:
epi_raw_forR.shape

In [ ]:
for col in epi_raw_forR.obs.columns:
    if col not in [ 'FemaleReproductiveAxis', 'sample']:
        del epi_raw_forR.obs[col]

In [ ]:
del epi_raw_forR.obsm

In [ ]:
%%R -i epi_raw_forR
epi_raw_forR

In [ ]:
%%R 
saveRDS(epi_raw_forR, '/lustre/scratch126/cellgen/team292/vl6/epi_mullerian_withpseudospace.rds')

In [ ]:
epi_raw.write('/nfs/team292/vl6/FetalReproductiveTract/epi_mullerian_withpseudospace_downsampled.h5ad')

## Fallopian axis

In [ ]:
epi_raw = sc.read('/nfs/team292/vl6/FetalReproductiveTract/epi_mullerian_withpseudospace.h5ad')

In [ ]:
epi_raw.obs['FemaleReproductiveAxis'].hist()

In [ ]:
epi_raw = epi_raw[[i <= 1 for i in epi_raw.obs['FemaleReproductiveAxis']]]

In [ ]:
epi_raw.obs['FemaleReproductiveAxis'].hist()

In [ ]:
epi_raw.shape

In [ ]:
epi_raw.X.toarray()[30:40, 30:40]

In [ ]:
epi_raw_forR = epi_raw.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in epi_raw_forR.var_names if not name.startswith('MT-')]
epi_raw_forR = epi_raw_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in epi_raw_forR.var_names if not name.startswith('RP')]
epi_raw_forR = epi_raw_forR[:, non_ribo_genes]

# remove heat shock protein genes
non_hps_genes = [name for name in epi_raw_forR.var_names if not name.startswith('HSP')]
epi_raw_forR = epi_raw_forR[:, non_hps_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in epi_raw_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
epi_raw_forR = epi_raw_forR[:, non_haemo_genes]

sc.pp.filter_genes(epi_raw_forR, min_cells=10)
sc.pp.filter_genes(epi_raw_forR, min_counts=1)

In [ ]:
for col in epi_raw_forR.obs.columns:
    if col not in [ 'FemaleReproductiveAxis', 'sample']:
        del epi_raw_forR.obs[col]

In [ ]:
del epi_raw_forR.obsm

In [ ]:
%%R -i epi_raw_forR
epi_raw_forR

In [ ]:
%%R 
saveRDS(epi_raw_forR, '/lustre/scratch126/cellgen/team292/vl6/epi_fallopian_withpseudospace.rds')